In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.plots import *
from torchtext.data import Field
from fastai.lm_rnn import seq2seq_reg
from quicknlp import SpacyTokenizer, print_batch, HierarchicalModelData
from pathlib import Path

In [2]:
!ls dataset/dialogue/

train


In [3]:
ubuntu_data = pd.read_csv("./dataset/dialogue/train/dialogueText.csv")

In [4]:
ubuntu_data.shape

(1038324, 6)

In [5]:
ubuntu_data.columns

Index(['folder', 'dialogueID', 'date', 'from', 'to', 'text'], dtype='object')

In [2]:
from torchtext.data import field
from torchtext.data import Example

In [25]:
f = Field(tokenize="spacy", lower="True")

In [9]:
twcs_data = pd.read_csv("./dataset/dialogue/twcs.csv")

In [10]:
twcs_data.shape

(1660645, 7)

In [12]:
twcs_data.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,75335,False,Tue Oct 31 02:03:43 +0000 2017,@75336 Hello! Can you DM your account email ad...,2,3.0
1,2,75336,True,Tue Oct 31 02:09:47 +0000 2017,@75335 I did!,NaN,1.0
2,3,75336,True,Tue Oct 31 02:02:09 +0000 2017,@75337 Got someone else's order in addition to...,1,NaN
3,4,75338,False,Tue Oct 31 02:26:51 +0000 2017,@75339 Our apologies. Please follow and DM wit...,NaN,5.0
4,5,75339,True,Tue Oct 31 01:03:38 +0000 2017,Hey so I’m at the @75340 Baltimore. They said ...,4,NaN
